## CellProfiler-OMERO demo
We have setup an example notebook to guide you through the steps to analise images stored in OMERO using CellProfiler

Let's start by importing some libraries we are going to need

In [1]:
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

from cp_omero_toolbox import *

import ezomero

import cellprofiler_core.preferences as cp_preferences

import pandas as pd
import tempfile


Bad key text.latex.preview in file /home/julio/miniconda3/envs/cellprofiler-core425/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.7.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/julio/miniconda3/envs/cellprofiler-core425/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.7.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /home/julio/miniconda3/envs/cellprofiler-core425/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 4

In [2]:
# Make CellProfiler run without a GUI
cp_preferences.set_headless()

# Tell CellProfiler to get input from and save output in a temp directory
output_dir = tempfile.TemporaryDirectory()
input_dir = tempfile.TemporaryDirectory()
cp_preferences.set_default_output_directory(output_dir.name)
cp_preferences.set_default_image_directory(input_dir.name)

Let's connect to OMERO. When we connect we get a connection object that we will have to use in every interaction with OMERO.

In [3]:
# Creating a connection object
host = "bioimage.france-bioinformatique.fr"
port = 4074
conn = ezomero.connect(host=host, port=port)

# Connecting
conn.connect()
# The connection will timeout after a period of inactivity. To avoid that we can tell our new connection to say "Hi, I'm still here"
conn.c.enableKeepAlive(60)
# Let's verify that we are connected
conn.isConnected()

True

Time to grasp a Dataset from OMERO and download a CellProfiler pipeline that is attached to it. Go to the browser, select a dataset and copy the ID.

In [4]:
dataset_id = int(input("Dataset id: "))


We create a new pipeline with that file, and we remove the first 4 modules. The first 4 modules are in charge of preparing the image data when they are loaded from disk. We don't need them here because we are using OMERO.


In [7]:
measurements_dfs = run_cp_pipeline(cp_pipeline_path=cp_pipeline_path,
                                   conn=conn,
                                   dataset_id=dataset_id,
                                   objects_to_image_table="Spots",
                                   objects_to_mask="Nuclei",
                                   objects_to_point="Spots",
                                   output_dir=output_dir)

Downloaded /tmp/tmpgnw71_ru/Megane_SpotInNuclei-encours_test_V3.cppipe
Remove module:  Images
Remove module:  Metadata
Remove module:  NamesAndTypes
Remove module:  Groups
Pipeline modules:
1 CorrectIlluminationCalculate
2 CorrectIlluminationCalculate
3 CorrectIlluminationApply
4 IdentifyPrimaryObjects
5 IdentifyPrimaryObjects
6 RelateObjects
7 MeasureObjectIntensity
8 RelateObjects
9 ConvertObjectsToImage
10 ConvertObjectsToImage
11 SaveImages
12 SaveImages


We can now start feeding images into the pipeline

In [8]:
cols = measurements_dfs["Image"].columns
vals = [measurements_dfs["Image"][c].values.tolist() for c in cols]

print(len(vals))
print(vals)



48
[[12.0, 25.0, 25.0, 24.0, 19.0], [238.0, 429.0, 521.0, 365.0, 603.0], [270.0, 694.0, 906.0, 738.0, 1204.0], [0.010000000000019327, 0.0, 0.0, 0.010000000000019327, 0.0], [0.0, 0.009999999999990905, 0.0, 0.0, 0.009999999999990905], [0.37000000000000455, 0.6100000000000136, 0.6099999999999852, 0.37999999999999545, 0.4300000000000068], [0.38999999999998636, 0.36999999999999034, 0.3499999999999943, 0.3599999999999852, 0.4399999999999977], [0.030000000000001137, 0.030000000000001137, 0.030000000000001137, 0.01999999999998181, 0.030000000000001137], [2.990000000000009, 3.0600000000000023, 3.069999999999993, 3.0, 3.0600000000000023], [2.0, 1.9300000000000068, 1.990000000000009, 2.0, 1.990000000000009], [0.37999999999999545, 0.38999999999998636, 0.37999999999999545, 0.3699999999999761, 0.37999999999999545], [0.14000000000001478, 0.15000000000000568, 0.14000000000001478, 0.14000000000001478, 0.13999999999998636], [0.04999999999998295, 0.060000000000002274, 0.04999999999998295, 0.0500000000000

In [10]:
# remove the output directory
output_dir.cleanup()

# and close the connection to the OMERO server
conn.close()